## Recycled code for bearings processing
- Need to make this into its own separate script 

In [19]:
import glob
import ast
import os, re
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [20]:
pattern = "./consolidated cracks/*.txt"
rows = []

DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}$")

In [21]:
for filepath in sorted(glob.glob(pattern)):
    filename = os.path.basename(filepath)
    date_str = filename.replace(".txt", "")
    year = date_str.split("-")[0]

    # Read file
    with open(filepath, "r") as f:
        file_content = f.read().strip()

    # Parse safely
    try:
        cracks_dict = ast.literal_eval(file_content)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        continue

    # Flatten cracks into rows
    for crack_name, coords_list in cracks_dict.items():
        for lat, lon in coords_list:
            rows.append([year, date_str, crack_name, lat, lon])

# Create DataFrame
df = pd.DataFrame(rows, columns=["year", "date", "crack", "lat", "lon"])

print(df.head())
print(f"Total rows: {len(df)}")

   year        date    crack        lat         lon
0  2014  2014-05-19  Crack 1  67.010664 -162.132102
1  2014  2014-05-19  Crack 1  67.008640 -162.129449
2  2014  2014-05-19  Crack 1  67.006616 -162.126797
3  2014  2014-05-19  Crack 1  67.004592 -162.124146
4  2014  2014-05-19  Crack 1  67.001028 -162.129183
Total rows: 4248


In [22]:
import math 

def segment_bearing_deg(lat1, lon1, lat2, lon2):
    """Initial great-circle bearing from (lat1,lon1) to (lat2,lon2), in [0,360)."""
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    Δλ = math.radians(lon2 - lon1)
    x = math.sin(Δλ) * math.cos(φ2)
    y = math.cos(φ1) * math.sin(φ2) - math.sin(φ1) * math.cos(φ2) * math.cos(Δλ)
    θ = math.degrees(math.atan2(x, y))
    return (θ + 360.0) % 360.0

In [23]:
from geopy import distance

grouped = df.groupby(["date", "crack"])
rows = []

for (date, crack), group in grouped:
    lats = group["lat"].tolist()
    lons = group["lon"].tolist()

    length = 0
    segment_lengths = []
    segment_bearings = []
    bearings = []

    for i in range(1, len(lats)):
        p1 = (lats[i-1], lons[i-1])
        p2 = (lats[i], lons[i])
        length += abs(distance.distance(p1,p2).km)
        segment_lengths.append(abs(distance.distance(p1,p2).km))
        segment_bearings.append(segment_bearing_deg(p1[0], p1[1], p2[0], p2[1]))
    # make axial
    for b in segment_bearings:
        b = b % 180
        if b < 0: # if less than zero reflect into first two quadrants
            b += 180
        if b > 90: # if greater than 90 degrees find deviation from true north
            b = 180 - b
        bearings.append(b)
    rows.append([date, crack, length, bearings])

lo = pd.DataFrame(rows, columns=["date", "crack", "length", "bearings"])
print(bearings)

[72.59821996173409, 72.59266607840902, 72.58711253022693, 72.5815593193741, 72.5760064456195, 72.57045390820204, 72.56490170783076, 72.55934984418167, 72.5537983179301, 72.5482471285004, 72.54269627624018]


## Begin Winds Processing

In [24]:
import xarray as xr
import numpy as np

# Open your three datasets
ds1 = xr.open_dataset("era5_2023_2025_06.grib", engine="cfgrib")
ds2 = xr.open_dataset("era5_2014_2023_05.grib", engine="cfgrib")
ds3 = xr.open_dataset("era5_2014_05.grib", engine="cfgrib")

# Concatenate along time dimension
ds = xr.concat([ds1, ds2, ds3], dim="time")

# Make sure time is sorted (important if ranges overlap)
ds = ds.sortby("time")

# Extract components
u = ds["u10"]
v = ds["v10"]

# Latitude weights
weights = np.cos(np.radians(ds["latitude"]))
weights.name = "weights"

# Weighted spatial mean, then resample to daily mean
u_daily = u.weighted(weights).mean(dim=("latitude", "longitude")).resample(time="1D").mean()
v_daily = v.weighted(weights).mean(dim=("latitude", "longitude")).resample(time="1D").mean()

# Convert to primary wind direction (degrees, meteorological convention)
primary_dir = (180 + np.degrees(np.arctan2(u_daily, v_daily))) % 360

# Put in dataframe
primary_dir = primary_dir.to_dataframe(name="primary_wind_dir_deg")

print(primary_dir)
#print("Number of daily values:", len(primary_dir))

ValueError: unrecognized engine cfgrib must be one of: ['scipy', 'store']

## Show wind dir frequency histogram

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 6))
colors = plt.cm.tab10.colors

primary_dir = primary_dir.copy()
primary_dir.index = pd.to_datetime(primary_dir.index)

all_angles = []

for date in primary_dir.index:
    wind_angle_deg = primary_dir.loc[date, "primary_wind_dir_deg"]
    if pd.isna(wind_angle_deg):
        continue

    wind_angle = np.radians(wind_angle_deg)

    # ensure numeric
    if np.isfinite(wind_angle):
        all_angles.append(wind_angle)

# convert list to numpy array
all_angles = np.array(all_angles)
print("Final array info:", all_angles.shape, "NaNs:", np.isnan(all_angles).sum())

# Define histogram parameters
n_bins = 24  # 15 degrees per bin
bins = np.linspace(0, 2*np.pi, n_bins + 1)

counts, _ = np.histogram(all_angles, bins=bins)
width = 2 * np.pi / n_bins

bars = ax.bar(
    bins[:-1], counts,
    width=width,
    align='edge',
    color=colors[0],
    edgecolor='k',
    alpha=0.7
)

# Optional: color scale by frequency
for bar, count in zip(bars, counts):
    bar.set_facecolor(plt.cm.viridis(count / counts.max() if counts.max() > 0 else 0))

# formatting
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_title("ERA5 Wind Direction Frequency by Day", va='bottom')

plt.show()

## Make KDE of diffs

In [ ]:
primary_dir = primary_dir.copy()
primary_dir.index = pd.to_datetime(primary_dir.index)

all_diffs = []

for date, group in lo.groupby("date"):
    # flatten bearings
    bearings = np.concatenate(group["bearings"].values)
    bearings = bearings[np.isfinite(bearings)]
    if len(bearings) == 0:
        continue

    bearings_rad = np.radians(bearings) # Convert to radians - all bearings <= 90 degrees

    date_ts = pd.to_datetime(date)
    if date_ts not in primary_dir.index:
        continue

    wind_angle_deg = primary_dir.loc[date_ts, "primary_wind_dir_deg"]
    if pd.isna(wind_angle_deg):
        continue
    wind_angle = np.radians(wind_angle_deg) # Convert wind angle to radians

    
    # Wind can be blowing at most 180 degrees from the crack bearing (since cracks are axial ??)
    diff = (bearings_rad - wind_angle) % np.pi

    # Reflect differences greater than 90° to their acute complement
    wind_diff = np.where(diff > np.pi / 2, np.pi - diff, diff)
    wind_diff = wind_diff[np.isfinite(wind_diff)]
    if len(wind_diff) == 0:
        continue

    wind_diff_axial = np.concatenate([wind_diff, wind_diff + np.pi])
    wind_diff_axial = wind_diff_axial[np.isfinite(wind_diff_axial)]
    if len(wind_diff_axial) > 0:
        all_diffs.append(wind_diff_axial)

# final concatenation
all_diffs = np.concatenate(all_diffs)
print(all_diffs)
print("Final array info:", all_diffs.shape, "NaNs:", np.isnan(all_diffs).sum())

kde = gaussian_kde(all_diffs)
theta = np.linspace(0, 2*np.pi, 180)
density = kde(theta)

ax.plot(theta, density, color=colors[i % len(colors)])

ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()